In [ ]:
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
import math
import csv
import statistics
import multiprocessing 
from multiprocessing import Pool
from collections import OrderedDict

In [3]:
snp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
symbols = list(snp500.Symbol)

In [4]:
def appendGrowth(lst):
    
    output = []
    
    for x in range(len(lst) - 1):
        
        try:
            diff = lst[x] - lst[x + 1]
            
            #if diff > 0:
            output.append(round((lst[x] - lst[x + 1]) / abs(lst[x + 1]) * 100, 2))
            #else:
            #    output.append(round((lst[x] - lst[x + 1]) / lst[x + 1] * 100, 2))
                
        except:
            continue
            
    return output


def getAvg(lst):
    
    summ = 0
    nums = 0
    
    for num in lst:
        if num != 0.0:
            summ += num
            nums += 1
    
    try:
        return round(summ / nums, 2)
    except:
        return 0 


def normalize(vals):

    tmp =  [vals[0], vals[1], vals[2]]

    med = statistics.median(tmp)

    if sum(tmp) != 0:

        normed = [i / sum(tmp) for i in tmp]

        for i, num in enumerate(normed):

            if num > 0.70:
                tmp[i] = med * 1.5


    return tmp


def getPricePerformance(symbol, period):

    stock = yf.Ticker(symbol)
    x = stock.history(period=period)
    last = x.iloc[0]['Close']
    curr = x.iloc[-1]['Close']

    percentDiff = round((curr - last) / abs(last) * 100, 2)

    if math.isnan(percentDiff): return 0
    
    return percentDiff


print(appendGrowth([100, -300, -200, 20]))

[133.33, -50.0, -1100.0]


In [5]:
def getCashFlowScore(symbol):

    stockScores = []

    operCash = []
    capex = []
    fcf = []
    investments = []

    score = 0
    investmentsAvgGrowth = 0
    operCashAvgGrowth = 0
    fcfAvgGrowth = 0
    stock = ''
    
    stockVals = []


    try:

        stock = yf.Ticker(symbol)

        print(symbol)

        for i, x in enumerate(list(stock.cashflow.itertuples())):
            
            if x[0] == 'Investments':
                for num in list(x)[1:]: investments.append(num)

            if x[0] == 'Total Cash From Operating Activities':
                for num in list(x)[1:]: operCash.append(num)

            if x[0] == 'Capital Expenditures':
                for num in list(x)[1:]: capex.append(num)


        for x, y in zip(operCash, capex):

            fcf.append(x - y)


        investmentsGrowth = appendGrowth(investments)
        operCashGrowth = appendGrowth(operCash)
        fcfGrowth = appendGrowth(fcf)

        investmentsAvgGrowth = getAvg(investmentsGrowth)
        operCashAvgGrowth = getAvg(operCashGrowth)
        fcfAvgGrowth = getAvg(fcfGrowth)


        stockVals = [investmentsAvgGrowth, operCashAvgGrowth, fcfAvgGrowth]

    except:
        print(symbol, 'err')


    if any([math.isnan(x) for x in stockVals]):
        
        return [symbol, 0, 0, 0, 0]
    
    else:

        tmp = normalize([investmentsAvgGrowth, operCashAvgGrowth, fcfAvgGrowth])
        
        investmentsAvgGrowth = tmp[0]
        operCashAvgGrowth = tmp[1]
        fcfAvgGrowth = tmp[2]

        score = investmentsAvgGrowth + operCashAvgGrowth + (fcfAvgGrowth * 2) 

        stockScores = [symbol, round(score, 2), investmentsAvgGrowth, operCashAvgGrowth, fcfAvgGrowth]

        return stockScores


In [6]:
print(normalize([5000 ,-59.92, 10.45]))

[15.674999999999999, -59.92, 10.45]


In [7]:
def getIncomeStatementScore(symbol):

    stockScores = []

    netIncome = []
    revenues = []
    opIncome = []
    grossProfit = []

    grossMargin = []
    opMargin = []
    netMargin = []

    score = 0
    stock = ''


    stock = yf.Ticker(symbol)


    for i, x in enumerate(list(stock.financials.itertuples())):

        if x[0] == 'Net Income':
            for num in list(x)[1:]: netIncome.append(num)

        if x[0] == 'Total Revenue':
            for num in list(x)[1:]: revenues.append(num)

        if x[0] == 'Operating Income':
            for num in list(x)[1:]: opIncome.append(num)

        if x[0] == 'Gross Profit':
            for num in list(x)[1:]: grossProfit.append(num)


    for x, y in zip(grossProfit, revenues):

        grossMargin.append(round(x / y * 100, 2))

    for x, y in zip(opIncome, revenues):

        opMargin.append(round(x / y * 100, 2))

    for x, y in zip(netIncome, revenues):

        netMargin.append(round(x / y * 100, 2))


    grossMarginGrowth = appendGrowth(grossMargin)
    opMarginGrowth = appendGrowth(opMargin)
    netMarginGrowth = appendGrowth(netMargin)

    grossMarginAvgGrowth = getAvg(grossMarginGrowth)
    opMarginAvgGrowth = getAvg(opMarginGrowth)
    netMarginAvgGrowth = getAvg(netMarginGrowth)

    stockVals = [grossMarginAvgGrowth, opMarginAvgGrowth, netMarginAvgGrowth]

    if any([math.isnan(x) for x in stockVals]):
        
        return [symbol, 0, 0, 0, 0]
        
    else:

        tmp = normalize([grossMarginAvgGrowth, opMarginAvgGrowth, netMarginAvgGrowth])
        
        '''
        print(grossMargin)
        print(opMargin)
        print(netMargin)

        print()
        print(grossMarginAvgGrowth, grossMarginGrowth)
        print(opMarginAvgGrowth, opMarginGrowth)
        print(netMarginAvgGrowth, netMarginGrowth)
        '''

        grossMarginAvgGrowth = tmp[0]
        opMarginAvgGrowth = tmp[1]
        netMarginAvgGrowth = tmp[2]


        score = (grossMarginAvgGrowth + opMarginAvgGrowth + netMarginAvgGrowth) * 5

        return [symbol, round(score, 2), grossMarginAvgGrowth, opMarginAvgGrowth, netMarginAvgGrowth]


getIncomeStatementScore('BA')

['BA', -2666.78, -69.93, -185.37, -278.055]

In [8]:
def getBalanceSheetScore(symbol):

    stockScores = []

    assets = []
    liabilities = []
    cash = []

    score = 0
    stock = ''

    stock = yf.Ticker(symbol)

    for i, x in enumerate(list(stock.balancesheet.itertuples())):

        if x[0] ==  'Total Current Assets':
            for num in list(x)[1:]: assets.append(num)

        if x[0] == 'Total Current Liabilities':
            for num in list(x)[1:]: liabilities.append(num)

        if x[0] == 'Cash':
            for num in list(x)[1:]: cash.append(num)



    assetsGrowth = appendGrowth(assets)
    liabGrowth = appendGrowth(liabilities)
    cashGrowth = appendGrowth(cash)

    assetsAvgGrowth = getAvg(assetsGrowth)
    liabAvgGrowth = getAvg(liabGrowth)
    cashAvgGrowth = getAvg(cashGrowth)


    stockVals = [assetsAvgGrowth, liabAvgGrowth, cashAvgGrowth]

    if any([math.isnan(x) for x in stockVals]):
        
        return [symbol, 0, 0, 0, 0]

    else:

        '''
        print(assets)
        print(liabilities)
        print(cash)
        print(assetsAvgGrowth, liabAvgGrowth, cashAvgGrowth)
        '''


        tmp = normalize([assetsAvgGrowth, liabAvgGrowth, cashAvgGrowth])

        assetsAvgGrowth = tmp[0]
        liabAvgGrowth = tmp[1]
        cashAvgGrowth = tmp[2]

        score = ((assetsAvgGrowth - liabAvgGrowth) * 5 + cashAvgGrowth) * 5

        return [symbol, round(score, 2), assetsAvgGrowth, liabAvgGrowth, cashAvgGrowth]


getBalanceSheetScore('BA')

['BA', 63.77, 9.135, 6.09, -2.47]

In [9]:
def getStockScores(symbol):

    stockScores = [] 

    cashFlowScores = getCashFlowScore(symbol)

    incomeStatementScore = getIncomeStatementScore(symbol)

    balanceSheetScore = getBalanceSheetScore(symbol)

    combScore = round(cashFlowScores[1] + incomeStatementScore[1] + balanceSheetScore[1], 2)

    oneYearPerf = 0 
    twoYearPerf = 0
    fiveYearPerf = 0

    try:
        oneYearPerf = getPricePerformance(symbol, '1y')
        twoYearPerf = getPricePerformance(symbol, '2y')
        fiveYearPerf = getPricePerformance(symbol, '5y')

    except:
        print(symbol, 'err')

    return OrderedDict({'Symbol': symbol, 'CombScore': combScore, 'AvgInvestmentsGrowth': cashFlowScores[2], 'AvgOperatingCashGrowth': cashFlowScores[3], 'AvgFCFGrowth': cashFlowScores[4], 
                        'AvgGrossMarginGrowth': incomeStatementScore[2], 'AvgOpMarginGrowth': incomeStatementScore[3], 'AvgNetMarginGrowth': incomeStatementScore[4],
                        'AvgAssetsGrowth': balanceSheetScore[2], 'AvgLiabGrowth': balanceSheetScore[3], 'AvgCashGrowth': balanceSheetScore[4], 
                        '1Y Performance': oneYearPerf, '2Y Performance': twoYearPerf, '5Y Performance': fiveYearPerf})

In [ ]:
p = Pool(100)

output = list(p.map(getStockScores, symbols))

p.close()
p.join()

In [ ]:
#output.sort(key = lambda l : l[0][1], reverse = True)
output.sort(key = lambda l : l['CombScore'], reverse = True)
output

In [ ]:
def exportData(fileName):

    MyFile = open(fileName, 'w')

    header = "Symbol, CombScore, AvgInvestmentsGrowth, AvgOperatingCashGrowth, AvgFCFGrowth, AvgGrossMarginGrowth, AvgOpMarginGrowth, AvgNetMarginGrowth, AvgAssetsGrowth, AvgLiabGrowth, AvgCashGrowth, 1Y Performance, 2Y Performance, 5Y Performance\n"

    MyFile.write(header)

    for row in output:

        rowVals = []
        stock = ''

        for key, value in row.items():

            rowVals.append(value)

        print(rowVals)
        MyFile.write(str(rowVals).replace('[', '').replace(']', '').replace("'", ''))
        MyFile.write('\n')

    
    MyFile.close()

    print('Saved as:', fileName)



exportData('S&P500Scoresv2.csv')